In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io as sio
import keras
import keras.backend as K

from sklearn.model_selection import ParameterGrid
from py_vollib import black_scholes_merton as bsm
from progressbar import ProgressBar
from scipy.stats import gamma
from scipy.stats import beta
from scipy.stats import uniform
from keras.models import Model
from keras.layers import Input, Dense
from sklearn.model_selection import train_test_split

import scipy.sparse
from mpl_toolkits.mplot3d import Axes3D

import import_ipynb

from FDC_self import FDExplicitEu

# Data Random Generation

In [ ]:
# S (spot price)
# gamma
def thisS(q):
    return gamma.ppf(q, a = 100, scale = 1)

# K (strike price)
# uniform (lower = 50, upper = 200)
def thisK(q):
    return uniform.ppf(q, 50, 200)

# (interest rate)
# uniform (lower = 0.01, upper = 0.18)
def thisR(q):
    return uniform.ppf(q, 0.01, 0.18)


# D (dividend)
# uniform (lower = 0.01, upper = 0.18)
def thisD(q):
    return 0
    # return uniform.ppf(q, 0.01, 0.18)

# t (time-to-maturity)
# t will be 3, 6, 9, 12 months for all examples (0.25, 0.5, 0.75, 1 year)

# sigma (volatility)
# beta (add small amount so volatility cannot be zero)
def thisSigma(q):
    return (beta.ppf(q, a = 2, b = 5) + 0.001)



# Control the total number of data
num_increment = 12
percentiles = pd.Series(np.linspace(0.01, 0.99, num_increment))
print(percentiles)



# Generation
S = percentiles.apply(thisS).to_numpy()
# print(S)
K = percentiles.apply(thisK).to_numpy()
q = percentiles.apply(thisD).to_numpy()
t = np.array([.25, .5, .75, 1])
r = percentiles.apply(thisR).to_numpy()
sigma = percentiles.apply(thisSigma).to_numpy()

# Initial data assigns

In [ ]:
S = 50          # spot price / asset price at time 0
K = 50          # strike price
q = 0
t = 1           # 1 year
r = 0.035       # australia 10Y government bond
Smax = int(5*K) # boundary asset price
M = 100         # asset discretize size
N = 1000        # time discretize size
is_call = True  # option type

In [ ]:
param_grid = {'S': S, 'K' : K, 'q' : q, 't' : t, 'r' : r, 'sigma' : sigma}
grid = ParameterGrid(param_grid)